<h1 align="center">Youtube data construction</h1>
<h2 align=center> What did I watch on youtube ? </h2>
<br>
<center>The purpose of this script is to create differents dataframe containing informations about my youtube watch history.</center>
<center>A second notebook will be used to analyse this data.</center>

The purpose here is to : 
* fetch and clean the download my-youtube-watch-history received from google service, 
* web scraping of video details and channel infos from the Youtube API. 

In [13]:
import pandas as pd
from googleapiclient.discovery import build

<h2 align="center">Raw Data</h2>

In [15]:
raw_data = pd.read_json('watch-history.json')

In [17]:
raw_data.to_excel('raw_data.xlsx')

<h2 align="center">Cleaning of data</h2>

<h3> columns cleaning </h3>

In [19]:
wt = raw_data.copy()
wt = wt.drop(['header','products','activityControls','description'],axis=1)

In [21]:
wt.head(3)

,title,titleUrl,subtitles,time,details
0,Vous avez regardé Récupérer ses Données You...,https://www.youtube.com/watch?v=4Vo6kF4jWn8,"[{'name': 'Labo Des Réseaux', 'url': 'https://...",2024-03-31T14:22:02.374Z,NaN
1,Vous avez regardé FR_fr_1019_Spring_Instream_Y...,https://www.youtube.com/watch?v=JR9tdqcj0-4,NaN,2024-03-31T14:21:52.118Z,[{'name': 'Des annonces Google'}]
2,Vous avez regardé BeautifulSoup + Requests | W...,https://www.youtube.com/watch?v=bargNl2WeN4,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31T13:52:12.369Z,NaN


<h3> columns organization </h3>

In [23]:
#Cleaning Title
wt['title'] = wt['title'].str.replace('Vous avez regardé ','')
#Fetch video-ID from the url and place it on a different columns
wt['video_ID'] = wt['titleUrl'].str.split('=').str[1]

wt.head(3)

,title,titleUrl,subtitles,time,details,video_ID
0,Récupérer ses Données YouTube,https://www.youtube.com/watch?v=4Vo6kF4jWn8,"[{'name': 'Labo Des Réseaux', 'url': 'https://...",2024-03-31T14:22:02.374Z,NaN,4Vo6kF4jWn8
1,FR_fr_1019_Spring_Instream_YT_16x9_10_C_414081...,https://www.youtube.com/watch?v=JR9tdqcj0-4,NaN,2024-03-31T14:21:52.118Z,[{'name': 'Des annonces Google'}],JR9tdqcj0-4
2,BeautifulSoup + Requests | Web Scraping in Python,https://www.youtube.com/watch?v=bargNl2WeN4,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31T13:52:12.369Z,NaN,bargNl2WeN4


In [25]:
#Fetch the date and time in different columns and set them to datetime type
wt['time'] = wt['time'].str.replace('T',' ').str.split('.').str[0]
wt['time'] = wt['time'].str.replace('Z','')
wt['time'] = pd.to_datetime(wt['time'])

<h3>Filter the advertisement in a different dataframe</h3>

* Dataframe tableau_pub : only Google advertisement
* Dataframe working_table (wt) : extract the advertisement

In [27]:
tableau_pub = wt.copy()
tableau_pub = wt[wt['subtitles'].isnull()&wt['details'].notnull()]
tableau_pub = tableau_pub.drop(['video_ID'],axis=1)
tableau_pub.head(3)

,title,titleUrl,subtitles,time,details
1,FR_fr_1019_Spring_Instream_YT_16x9_10_C_414081...,https://www.youtube.com/watch?v=JR9tdqcj0-4,NaN,2024-03-31 14:21:52,[{'name': 'Des annonces Google'}]
3,02 ResponsableProjetCybersecuriteSI 16 9,https://www.youtube.com/watch?v=wKLFC8Uxvtc,NaN,2024-03-31 13:52:03,[{'name': 'Des annonces Google'}]
4,20221219 HELLOFRESH PEACE OF MIND V5 ssST,https://www.youtube.com/watch?v=xDlM2D1LGDQ,NaN,2024-03-31 13:51:40,[{'name': 'Des annonces Google'}]


In [29]:
wt = wt[wt['subtitles'].notnull()]
wt = wt.drop(['details'],axis=1)
wt.head()

,title,titleUrl,subtitles,time,video_ID
0,Récupérer ses Données YouTube,https://www.youtube.com/watch?v=4Vo6kF4jWn8,"[{'name': 'Labo Des Réseaux', 'url': 'https://...",2024-03-31 14:22:02,4Vo6kF4jWn8
2,BeautifulSoup + Requests | Web Scraping in Python,https://www.youtube.com/watch?v=bargNl2WeN4,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31 13:52:12,bargNl2WeN4
5,Inspecting Web Pages with HTML | Web Scraping ...,https://www.youtube.com/watch?v=q-kbzWjyPak,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31 13:43:46,q-kbzWjyPak
7,Building an Automated File Sorter in File Expl...,https://www.youtube.com/watch?v=gs0FNQR0njI,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31 13:39:53,gs0FNQR0njI
8,FREE Data Analyst Bootcamp!!,https://www.youtube.com/watch?v=rGx1QNdYzvs,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31 13:39:23,rGx1QNdYzvs


<h3> Fetch data from the subtitles columns </h3>

In [31]:
author = []
channel_url = []
for element in wt['subtitles']:
    author.append(element[0]["name"])
    channel_url.append(element[0]["url"])
wt['creator']=author
wt['channel_url']=channel_url

In [33]:
wt.head(3)

,title,titleUrl,subtitles,time,video_ID,creator,channel_url
0,Récupérer ses Données YouTube,https://www.youtube.com/watch?v=4Vo6kF4jWn8,"[{'name': 'Labo Des Réseaux', 'url': 'https://...",2024-03-31 14:22:02,4Vo6kF4jWn8,Labo Des Réseaux,https://www.youtube.com/channel/UCMJEpQVv0p3qz...
2,BeautifulSoup + Requests | Web Scraping in Python,https://www.youtube.com/watch?v=bargNl2WeN4,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31 13:52:12,bargNl2WeN4,Alex The Analyst,https://www.youtube.com/channel/UC7cs8q-gJRlGw...
5,Inspecting Web Pages with HTML | Web Scraping ...,https://www.youtube.com/watch?v=q-kbzWjyPak,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31 13:43:46,q-kbzWjyPak,Alex The Analyst,https://www.youtube.com/channel/UC7cs8q-gJRlGw...


<h3> Index reset </h3>

In [ ]:
wt.index=range(len(wt))
wt = wt.drop(['subtitles'],axis=1)
wt.head(3)

<h3>Fetch channel_id from the channel_url </h3>

In [41]:
wt['channel_id']= wt['channel_url'].str.replace('https://www.youtube.com/channel/','')
wt.head(3)

,title,titleUrl,time,video_ID,creator,channel_url,channel_id
0,Récupérer ses Données YouTube,https://www.youtube.com/watch?v=4Vo6kF4jWn8,2024-03-31 14:22:02,4Vo6kF4jWn8,Labo Des Réseaux,https://www.youtube.com/channel/UCMJEpQVv0p3qz...,UCMJEpQVv0p3qzJnQ9IuiiMA
1,BeautifulSoup + Requests | Web Scraping in Python,https://www.youtube.com/watch?v=bargNl2WeN4,2024-03-31 13:52:12,bargNl2WeN4,Alex The Analyst,https://www.youtube.com/channel/UC7cs8q-gJRlGw...,UC7cs8q-gJRlGwj4A8OmCmXg
2,Inspecting Web Pages with HTML | Web Scraping ...,https://www.youtube.com/watch?v=q-kbzWjyPak,2024-03-31 13:43:46,q-kbzWjyPak,Alex The Analyst,https://www.youtube.com/channel/UC7cs8q-gJRlGw...,UC7cs8q-gJRlGwj4A8OmCmXg


<h2 align=center> Recuperation channels data from Youtube API </h2>

* identification channels where I watched more than 10 videos
* convert there channel_id to a list
* pass the list to the youtube API and scrap their data

In [43]:
count = wt.channel_id.value_counts()
count = count[count > 10]
len(count)

312

In [45]:
channel_scraping = pd.DataFrame(data=count)
channel_scraping = channel_scraping.reset_index()

In [47]:
api_key ='****'
youtube= build('youtube', 'v3', developerKey=api_key)

In [49]:
channel_ids = channel_scraping['channel_id'].tolist()

In [51]:
def get_channel_stats(youtube, channel_ids):
    all_data = []
    
    for i in range(0,len(channel_ids),50):                  
        request = youtube.channels().list(
            part="snippet,contentDetails,statistics,id",
            id = ','.join(channel_ids[i:i+50]))
        response = request.execute()

        for i in range(len(response['items'])):
            data =dict(Channel_name = response['items'][i]['snippet']['title'],
                       channel_id = response['items'][i]['id'], 
                       Channel_date = response['items'][i]['snippet']['publishedAt'],
                       Subscribers = response['items'][i]['statistics']['subscriberCount'],
                       Views = response['items'][i]['statistics']['viewCount'],
                       country = response['items'][i]['snippet'].get('country'),
                       Total_videos = response['items'][i]['statistics']['videoCount'])
                       
            all_data.append(data)
    
    return all_data

In [53]:
channel_statitics = get_channel_stats(youtube, channel_ids)
channel_data = pd.DataFrame(channel_statitics)

In [55]:
data_channel = pd.merge(channel_data, channel_scraping, how='inner',on='channel_id')
data_channel = data_channel.sort_values('count',ascending=False)
data_channel

,Channel_name,channel_id,Channel_date,Subscribers,Views,country,Total_videos,count
7,CloudKid,UCSa8IUd1uEjlREMa21I3ZPQ,2013-07-30T20:10:22Z,5330000,2972232344,US,2758,1432
16,ImagineDragonsVEVO,UCpx_k19S2vUutWUUM9qmXEg,2012-01-23T18:04:00Z,9520000,14902187298,None,226,401
15,Adam Jensen,UChRJQ2QyKsY1EbBqM8GzzRg,2011-10-07T00:27:55Z,118000,90312820,US,71,330
44,McSkyz,UC4OjGShlHwNNd6jQ23GVJ5A,2011-07-18T09:49:59Z,1750000,282601923,FR,288,234
17,twenty one pilots,UCBQZwaNPFfJ1gZ1fLZpAEGw,2009-09-05T05:57:18Z,13400000,5778309370,US,205,193
...,...,...,...,...,...,...,...,...
276,P2 DBOXS,UCgO3YEcCq98JZVF672wX0lg,2023-03-25T19:36:09.997693Z,313000,335120702,BD,93,11
275,POPSLAY,UCZNu2DGg9phKQUzUywxawAA,2021-04-18T17:00:25.318517Z,129000,12277186,FR,83,11
253,JokerTwd,UC0i-1mpARwzftXp7DdpKDOw,2015-04-11T09:03:14Z,50100,39607415,FR,416,11
257,Anthony Crazy,UCwlLTQ2yiKu0GCwNK70-pww,2018-04-02T20:50:23Z,374000,174084760,FR,69,11


In [57]:
# cleanning datatype : numeric
numeric_cols = ['Subscribers','Views','Total_videos']
data_channel[numeric_cols] = data_channel[numeric_cols].apply(pd.to_numeric, errors='coerce')

#cleaning datatype: datetime
data_channel['Channel_date'] = data_channel['Channel_date'].str.replace('T',' ').str.split('.').str[0]
data_channel['Channel_date'] = data_channel['Channel_date'].str.replace('Z','')
data_channel['Channel_date'] = pd.to_datetime(data_channel['Channel_date'])


#result
data_channel.dtypes

Channel_name            object
channel_id              object
Channel_date    datetime64[ns]
Subscribers              int64
Views                    int64
country                 object
Total_videos             int64
count                    int64
dtype: object

In [59]:
data_channel.head(3)

,Channel_name,channel_id,Channel_date,Subscribers,Views,country,Total_videos,count
7,CloudKid,UCSa8IUd1uEjlREMa21I3ZPQ,2013-07-30 20:10:22,5330000,2972232344,US,2758,1432
16,ImagineDragonsVEVO,UCpx_k19S2vUutWUUM9qmXEg,2012-01-23 18:04:00,9520000,14902187298,None,226,401
15,Adam Jensen,UChRJQ2QyKsY1EbBqM8GzzRg,2011-10-07 00:27:55,118000,90312820,US,71,330


<h2 align = center> Recuperation datas from all my videos history </h2>

### Recuperation data youtube's API

In [61]:
count_video = wt.video_ID.value_counts()
len(count_video)

16648

In [63]:
video_scraping = pd.DataFrame(data=count_video)
video_scraping = video_scraping.reset_index()
video_scraping.head(3)

,video_ID,count
0,EgwQG3MYp3o,67
1,i4AXVFggAnk,64
2,L3wKzyIN1yk,63


In [65]:
video_ids = video_scraping['video_ID'].tolist()

In [67]:
def get_video_details(youtube, video_ids):
    all_videos_stats = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part = 'snippet, statistics',
            id=','.join(video_ids[i:i+50]))
        response = request.execute()

        for video in response['items']:
            video_stats = dict(title = video['snippet']['title'],
                                Published_date = video['snippet']['publishedAt'],
                                Views = video['statistics'].get('viewCount'),
                                Like = video['statistics'].get('likeCount'),
                                Comments = video['statistics'].get('commentCount'),
                                Videocategory = video['snippet']['categoryId']
                              )
            all_videos_stats.append(video_stats)

    return all_videos_stats

In [69]:
video_statitics = get_video_details(youtube, video_ids)
video_data = pd.DataFrame(video_statitics)
video_data.head(3)

,title,Published_date,Views,Like,Comments,Videocategory
0,Silent Child - F**k You (Lyric Video),2020-08-26T15:27:16Z,42422990,598706,3367,10
1,Stela Cole - Love Like Mine,2020-09-25T15:27:08Z,17870692,140165,933,10
2,Rag'n'Bone Man - Human (Official Video),2016-07-21T11:00:00Z,1884737908,12215869,248117,10


In [71]:
data_video = pd.merge(video_data, wt, how='inner',on='title')
data_video.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22238 entries, 0 to 22237
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           22238 non-null  object        
 1   Published_date  22238 non-null  object        
 2   Views           22237 non-null  object        
 3   Like            21969 non-null  object        
 4   Comments        22076 non-null  object        
 5   Videocategory   22238 non-null  object        
 6   titleUrl        22238 non-null  object        
 7   time            22238 non-null  datetime64[ns]
 8   video_ID        22238 non-null  object        
 9   creator         22238 non-null  object        
 10  channel_url     22238 non-null  object        
 11  channel_id      22238 non-null  object        
dtypes: datetime64[ns](1), object(11)
memory usage: 2.0+ MB


### formating dataframe

In [73]:
# cleanning datatype : numeric
numeric_cols = ['Views','Like','Comments']
data_video[numeric_cols] = data_video[numeric_cols].apply(pd.to_numeric, errors='coerce')

#cleaning datatype: datetime
data_video['Published_date'] = data_video['Published_date'].str.split('T').str[0]
data_video['Published_date'] = pd.to_datetime(data_video['Published_date'])

#result
data_video.dtypes

title                     object
Published_date    datetime64[ns]
Views                    float64
Like                     float64
Comments                 float64
Videocategory             object
titleUrl                  object
time              datetime64[ns]
video_ID                  object
creator                   object
channel_url               object
channel_id                object
dtype: object

In [75]:
data_video.Videocategory.value_counts()

Videocategory
10    6143
24    5244
22    4172
23    1912
20    1199
27    1000
1      854
26     558
28     356
25     345
17     230
19     126
15      53
2       26
29      13
44       7
Name: count, dtype: int64

In [77]:
my_categories = {'10':'Music','24':'Entertainment','22':'People&Blog','23':'Comedy','20':'Gaming','27':'Education',
                 '1':'Film&Animation','26':'HowTo&Style','28':'Science&Technology','25':'News&Politics','17':'Sports',
                '19':'Travel&Events','15':'Pets&Animal','2':'Autos','29':'NonProfits&Activism','44':'Trailers'}

In [79]:
data_video['Videocategory'] = data_video['Videocategory'].map(my_categories)

In [81]:
data_video.Videocategory.value_counts()

Videocategory
Music                  6143
Entertainment          5244
People&Blog            4172
Comedy                 1912
Gaming                 1199
Education              1000
Film&Animation          854
HowTo&Style             558
Science&Technology      356
News&Politics           345
Sports                  230
Travel&Events           126
Pets&Animal              53
Autos                    26
NonProfits&Activism      13
Trailers                  7
Name: count, dtype: int64

### 2018-2023

In [104]:
data_video.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22238 entries, 0 to 22237
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           22238 non-null  object        
 1   Published_date  22238 non-null  datetime64[ns]
 2   Views           22237 non-null  float64       
 3   Like            21969 non-null  float64       
 4   Comments        22076 non-null  float64       
 5   Videocategory   22238 non-null  object        
 6   titleUrl        22238 non-null  object        
 7   time            22238 non-null  datetime64[ns]
 8   video_ID        22238 non-null  object        
 9   creator         22238 non-null  object        
 10  channel_url     22238 non-null  object        
 11  channel_id      22238 non-null  object        
 12  year            22238 non-null  int32         
dtypes: datetime64[ns](2), float64(3), int32(1), object(7)
memory usage: 2.1+ MB


In [106]:
data_video['year'] = data_video['time'].dt.year

nb_video_2017 = len(data_video[(data_video['year']<=2017)])
nb_video_2018 = len(data_video[(data_video['year']>2017)])

print(f'nb videos 2012 to 2017 : {nb_video_2017}')
print(f'nb videos 2018 to 2024 : {nb_video_2018}')

nb videos 2012 to 2017 : 1139
nb videos 2018 to 2024 : 21099


In [108]:
# filtrer le plus de données, enlevé 2024, data sur année entière
data_video_2018 = data_video.copy()
data_video_2018 = data_video_2018[(data_video_2018['year']>2017) & (data_video_2018['year']<2024)]
data_video_2018.groupby('year').size()

year
2018     1723
2019      365
2020      610
2021     1245
2022     2693
2023    11935
dtype: int64

### fetch duration of videos

In [ ]:
#excel transitoire pour récolter les durées manuellement (irrécupérable via API ou Selene) +suppr videos, watch only the begining ... 
df_video_id= pd.DataFrame(video_data[['title','titleUrl','video_ID','year']])
df_video_id.to_excel('vd_id_time.xlsx')

In [98]:
#retrieve form excel avec durée en minute
df_duration = pd.read_excel('vd_id_time_duration.xlsx')
df_duration= df_duration.drop(["Colonne1"],axis=1)
df_duration

,title,titleUrl,video_ID,year,duration
0,Silent Child - F**k You (Lyric Video),https://www.youtube.com/watch?v=EgwQG3MYp3o,EgwQG3MYp3o,2023,3.30
1,Silent Child - F**k You (Lyric Video),https://www.youtube.com/watch?v=EgwQG3MYp3o,EgwQG3MYp3o,2023,3.30
2,Silent Child - F**k You (Lyric Video),https://www.youtube.com/watch?v=EgwQG3MYp3o,EgwQG3MYp3o,2023,3.30
3,Silent Child - F**k You (Lyric Video),https://www.youtube.com/watch?v=EgwQG3MYp3o,EgwQG3MYp3o,2023,3.30
4,Silent Child - F**k You (Lyric Video),https://www.youtube.com/watch?v=EgwQG3MYp3o,EgwQG3MYp3o,2023,3.30
...,...,...,...,...,...
18626,imagine the day they see you again..,https://www.youtube.com/watch?v=ZrBZNonf_qw,ZrBZNonf_qw,2023,0.07
18627,Je suis une BDH ?! 🚨,https://www.youtube.com/watch?v=hIlfuwJ9Of0,hIlfuwJ9Of0,2023,0.22
18628,Aww this was so cute 🥹❤️,https://www.youtube.com/watch?v=i-bZBXNExik,i-bZBXNExik,2023,0.27
18629,IM JUST KEN #Shorts,https://www.youtube.com/watch?v=947RyXTZ8Yo,947RyXTZ8Yo,2023,0.14


In [114]:
df_duration.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18631 entries, 0 to 18630
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   title     18631 non-null  object 
 1   titleUrl  18631 non-null  object 
 2   video_ID  18631 non-null  object 
 3   year      18631 non-null  int64  
 4   duration  17756 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 727.9+ KB


In [112]:
data_video_2018.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18571 entries, 9 to 22236
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           18571 non-null  object        
 1   Published_date  18571 non-null  datetime64[ns]
 2   Views           18570 non-null  float64       
 3   Like            18319 non-null  float64       
 4   Comments        18451 non-null  float64       
 5   Videocategory   18571 non-null  object        
 6   titleUrl        18571 non-null  object        
 7   time            18571 non-null  datetime64[ns]
 8   video_ID        18571 non-null  object        
 9   creator         18571 non-null  object        
 10  channel_url     18571 non-null  object        
 11  channel_id      18571 non-null  object        
 12  year            18571 non-null  int32         
dtypes: datetime64[ns](2), float64(3), int32(1), object(7)
memory usage: 1.9+ MB


In [118]:
# merge df_duration et raw_data
video_data_duration = pd.merge(data_video_2018,df_duration,how='inner',on=['video_ID','title','titleUrl','year'])
video_data_duration.head(3)

,title,Published_date,Views,Like,Comments,Videocategory,titleUrl,time,video_ID,creator,channel_url,channel_id,year,duration
0,Silent Child - F**k You (Lyric Video),2020-08-26,42422990.0,598706.0,3367.0,Music,https://www.youtube.com/watch?v=EgwQG3MYp3o,2023-09-13 18:40:34,EgwQG3MYp3o,CloudKid,https://www.youtube.com/channel/UCSa8IUd1uEjlR...,UCSa8IUd1uEjlREMa21I3ZPQ,2023,3.3
1,Silent Child - F**k You (Lyric Video),2020-08-26,42422990.0,598706.0,3367.0,Music,https://www.youtube.com/watch?v=EgwQG3MYp3o,2023-09-13 18:40:34,EgwQG3MYp3o,CloudKid,https://www.youtube.com/channel/UCSa8IUd1uEjlR...,UCSa8IUd1uEjlREMa21I3ZPQ,2023,3.3
2,Silent Child - F**k You (Lyric Video),2020-08-26,42422990.0,598706.0,3367.0,Music,https://www.youtube.com/watch?v=EgwQG3MYp3o,2023-09-13 18:40:34,EgwQG3MYp3o,CloudKid,https://www.youtube.com/channel/UCSa8IUd1uEjlR...,UCSa8IUd1uEjlREMa21I3ZPQ,2023,3.3


<h2 align='center'> Save Data to Excel </h2>

In [120]:
with pd.ExcelWriter ("youtube_data.xlsx") as writer :
    wt.to_excel(writer, sheet_name='working_data',index=False)
    tableau_pub.to_excel(writer, sheet_name='pub_data', index=False)
    data_channel.to_excel(writer, sheet_name='channel_data',index=False)
    data_video.to_excel(writer,sheet_name='video_data',index=False)
    data_video_2018.to_excel(writer,sheet_name='video_data_filter_2018',index=False)
    video_data_duration.to_excel(writer,sheet_name='video_data_complete',index=False)